# Demostración de Threading con Estado Compartido

Este notebook demuestra cómo ejecutar dos hilos simultáneamente y manejar adecuadamente el estado de una variable compartida usando sincronización.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Usando dispositivo: {device}")
# Definir la arquitectura de la red neuronal convolucional
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Primera capa convolucional
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Capas de pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Capas fully connected
        self.fc1 = nn.Linear(128 * 3 * 3, 256)  # 28->14->7->3 después de 3 poolings
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)  # 10 clases para MNIST
        
        # Dropout para regularización
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # Aplicar convoluciones con ReLU y pooling
        x = self.pool(F.relu(self.conv1(x)))  # 28x28 -> 14x14
        x = self.pool(F.relu(self.conv2(x)))  # 14x14 -> 7x7
        x = self.pool(F.relu(self.conv3(x)))  # 7x7 -> 3x3
        
        # Aplanar para las capas fully connected
        x = x.view(-1, 128 * 3 * 3)
        
        # Capas fully connected
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=1)


class ConvNet2(nn.Module):
    def __init__(self):
        super(ConvNet2, self).__init__()
        # Primera capa convolucional
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        # Capas de pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Capas fully connected
        self.fc1 = nn.Linear(64 * 3 * 3, 256)  # 28->14->7->3 después de 3 poolings
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)  # 10 clases para MNIST
        
        # Dropout para regularización
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        # Aplicar convoluciones con ReLU y pooling
        x = self.pool(F.relu(self.conv1(x)))  # 28x28 -> 14x14
        x = self.pool(F.relu(self.conv2(x)))  # 14x14 -> 7x7
        x = self.pool(F.relu(self.conv3(x)))  # 7x7 -> 3x3
        
        # Aplanar para las capas fully connected
        x = x.view(-1, 128 * 3 * 3)
        
        # Capas fully connected
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=1)

# Crear el modelo convolucional
conv_model = ConvNet().to(device)
conv_optimizer = optim.Adam(conv_model.parameters(), lr=0.001)

print(f"Arquitectura del modelo convolucional:\n{conv_model}")
print(f"\nNúmero de parámetros: {sum(p.numel() for p in conv_model.parameters())}")

Usando dispositivo: cuda
Arquitectura del modelo convolucional:
ConvNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1152, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

Número de parámetros: 422026


Función para entrenar y evaluar el modelo

In [2]:
def train_and_evaluate(model, optimizer, train_loader, test_loader, epochs=5, thread_name="thread"):
    """
    Entrena y evalúa un modelo de red neuronal.
    
    Args:
        model (nn.Module): El modelo de red neuronal a entrenar
        optimizer (torch.optim.Optimizer): El optimizador para actualizar los parámetros
        train_loader (DataLoader): DataLoader con los datos de entrenamiento
        test_loader (DataLoader): DataLoader con los datos de prueba
        epochs (int): Número de épocas de entrenamiento (por defecto 5)
    
    Returns:
        None: La función imprime el progreso del entrenamiento y la precisión final
    """
    print(f"{thread_name}: Iniciando entrenamiento con {epochs} épocas...")
    model.train()
    
    # Ciclo de entrenamiento por épocas
    for epoch in range(epochs):
        total_loss = 0
        
        # Procesar cada batch de datos de entrenamiento
        for batch_idx, (data, target) in enumerate(train_loader):
            # Mover datos al dispositivo (GPU/CPU)
            data, target = data.to(device), target.to(device)
            
            # Limpiar gradientes del paso anterior
            optimizer.zero_grad()
            
            # Forward pass: calcular predicciones
            output = model(data)
            
            # Calcular la función de pérdida (negative log likelihood)
            loss = F.nll_loss(output, target)
            
            # Backward pass: calcular gradientes
            loss.backward()
            
            # Actualizar parámetros del modelo
            optimizer.step()
            
            # Acumular pérdida para mostrar progreso
            total_loss += loss.item()
            
            # # Mostrar progreso cada 100 batches
            # if batch_idx % 100 == 0:
            #     print(f"{thread_name}: Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}")

        # Mostrar pérdida promedio de la época
        avg_loss = total_loss / len(train_loader)
        print(f"{thread_name}: Epoch {epoch+1} completada - Loss promedio: {avg_loss:.4f}")

    # Configurar el modelo en modo de evaluación (desactiva dropout, etc.)
    model.eval()
    correct = 0
    
    # Evaluación sin calcular gradientes para ahorrar memoria
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            # Mover datos al dispositivo
            data, target = data.to(device), target.to(device)
            
            # Obtener predicciones del modelo
            output = model(data)
            
            # Obtener la clase predicha (índice con mayor probabilidad)
            pred = output.argmax(dim=1, keepdim=True)
            
            # Contar predicciones correctas
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            # Mostrar progreso cada 50 batches
            if batch_idx % 50 == 0:
                current_accuracy = 100. * correct / ((batch_idx + 1) * test_loader.batch_size)
                print(f"{thread_name}: Evaluando, Batch {batch_idx}, Accuracy: {current_accuracy:.2f}%")

    # Calcular y mostrar la precisión final
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"{thread_name}: Test set accuracy: {accuracy:.2f}%")

Creamos los dataloader

In [3]:
import torchvision

train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data', train=True, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize((0.1307,), (0.3081,))
                               ])),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('./data', train=False, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize((0.1307,), (0.3081,))
                               ])),
    batch_size=64, shuffle=False)

creamos 2 hilos y entrenamos y evaluamos el modelo a la vez

In [4]:
# Entrenar y evaluar el modelo convolucional
import threading

conv_model1 = ConvNet().to(device)
conv_optimizer1 = optim.Adam(conv_model1.parameters(), lr=0.001)

conv_model2 = ConvNet().to(device)
conv_optimizer2 = optim.Adam(conv_model2.parameters(), lr=0.001)

thread1 = threading.Thread(target=train_and_evaluate, args=(conv_model1, conv_optimizer1, train_loader, test_loader, 5, "Thread-1"))
thread2 = threading.Thread(target=train_and_evaluate, args=(conv_model2, conv_optimizer2, train_loader, test_loader, 5, "Thread-2"))

thread1.start()
thread2.start()

thread1.join()
thread2.join()


Thread-1: Iniciando entrenamiento con 5 épocas...
Thread-2: Iniciando entrenamiento con 5 épocas...
Thread-1: Epoch 1 completada - Loss promedio: 0.2581
Thread-1: Epoch 1 completada - Loss promedio: 0.2581
Thread-2: Epoch 1 completada - Loss promedio: 0.2800
Thread-2: Epoch 1 completada - Loss promedio: 0.2800
Thread-1: Epoch 2 completada - Loss promedio: 0.0770
Thread-1: Epoch 2 completada - Loss promedio: 0.0770
Thread-2: Epoch 2 completada - Loss promedio: 0.0788
Thread-2: Epoch 2 completada - Loss promedio: 0.0788
Thread-1: Epoch 3 completada - Loss promedio: 0.0567
Thread-1: Epoch 3 completada - Loss promedio: 0.0567
Thread-2: Epoch 3 completada - Loss promedio: 0.0572
Thread-2: Epoch 3 completada - Loss promedio: 0.0572
Thread-1: Epoch 4 completada - Loss promedio: 0.0480
Thread-1: Epoch 4 completada - Loss promedio: 0.0480
Thread-2: Epoch 4 completada - Loss promedio: 0.0479
Thread-2: Epoch 4 completada - Loss promedio: 0.0479
Thread-1: Epoch 5 completada - Loss promedio: 0.0392